In [1]:
############################
## Load required packages ##
############################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from covid19model.models.models import COVID19_SEIQRD_spatial_stratified_rescaling, jit_main_function_spatial
from covid19model.models.utils import initialize_COVID19_SEIQRD_stratified_vacc

# Import time-dependent parameter functions for resp. P, Nc, alpha, N_vacc, season_factor
from covid19model.models.time_dependant_parameter_fncs import make_mobility_update_function, \
                                                          make_contact_matrix_function, \
                                                          make_VOC_function, \
                                                          make_vaccination_rescaling_function, \
                                                          make_seasonality_function_NEW

# Import packages containing functions to load in data used in the model and the time-dependent parameter functions
from covid19model.data import mobility, sciensano, model_parameters, VOC

In [2]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : 5f648bf1706dd75a9ca0d29f26eadfbb595fe52b
python           : 3.7.3.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19041
machine          : AMD64
processor        : Intel64 Family 6 Model 158 Stepping 13, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.3.2
numpy            : 1.19.1
pytz             : 2021.3
dateutil         : 2.8.2
pip              : 21.2.4
setuptools       : 49.6.0
Cython           : 0.29.25
pytest           : 6.2.5
hypothesis       : None
sphinx           : 4.4.0
blosc            : None
feather          : None
xlsxwriter       : 3.0.2
lxml.etree       : 4.8.0
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.3
IPython          : 7.31.1
pandas_datareader: None
bs4              : 4.10.0
bottleneck       : 1.3.2
fsspec         

In [ ]:
#########################
## Load necessary data ##
#########################

# Population size, interaction matrices and the model parameters
initN, Nc_dict, params, CORE_samples_dict = model_parameters.get_COVID19_SEIQRD_parameters(spatial='prov')

# Google Mobility data (for social contact Nc)
df_google = mobility.get_google_mobility_data(update=False, provincial=True)

# Load and format mobility dataframe (for mobility place)
proximus_mobility_data = mobility.get_proximus_mobility_data('prov')

# Variants of concern
VOCs = ['WT', 'abc', 'delta']
VOC_logistic_growth_parameters, VOC_params = model_parameters.get_COVID19_SEIQRD_VOC_parameters(initN, params['h'], VOCs=VOCs)
params.update(VOC_params)

# Load and format local vaccination-induced rescaling data, which is also under the sciensano object
rescaling_df = sciensano.get_vaccination_rescaling_values(spatial=True)

In [ ]:
# Delete parameters that are superfluous for the new approach

superfluous_params = ['f_immune_escape', 'e_s', 'e_h', 'e_i', 'doses', 'd_vacc', 'N_vacc', 'daily_doses', 'delay_immunity', 'vacc_order', 'stop_idx', 'refusal', 'initN', 'default_mobility', 'onset_days']
for key in superfluous_params:
    if key in params:
        params.pop(key)
        
# Add new parameters (these will be time-dependent soon)
params['E_susc'] = np.ones([11, 10])
params['E_inf'] = np.ones([11, 10])
params['E_hosp'] = np.ones([11, 10])

# Add seasonality parameter
params['seasonality'] = 1

In [ ]:
##################################################
## Construct time-dependent parameter functions ##
##################################################

# Time-dependent social contact matrix over all policies, updating Nc
policy_function = make_contact_matrix_function(df_google, Nc_dict).policies_all_spatial
policy_function_work = make_contact_matrix_function(df_google, Nc_dict).policies_all_work_only

# Time-dependent mobility function, updating P (place)
mobility_function = \
    make_mobility_update_function(proximus_mobility_data).mobility_wrapper_func

# Time-dependent VOC function, updating alpha
VOC_function = make_VOC_function(VOC_logistic_growth_parameters)

# Time-dependent vaccination-induced rescaling parameter functions, updating E_susc^g, E_inf^g, E_hosp^g
E_susc_function = make_vaccination_rescaling_function(rescaling_df).E_susc
E_inf_function = make_vaccination_rescaling_function(rescaling_df).E_inf
E_hosp_function = make_vaccination_rescaling_function(rescaling_df).E_hosp

# Time-dependent seasonality function, updating season_factor
seasonality_function = make_seasonality_function_NEW()

In [ ]:
proximus_mobility_data[proximus_mobility_data.index.dayofweek < 5]['place'][pd.Timestamp(2021, 5, 1):pd.Timestamp(2021, 6, 30)].mean()

In [ ]:
####################
## Initial states ##
####################

import pickle
data_path = "../../data/"

# Get correct initial condition
samples_path = data_path + 'interim/model_parameters/COVID19_SEIQRD/initial_conditions/prov/'

with open(samples_path+'initial_states-COVID19_SEIQRD_spatial.pickle', 'rb') as handle:
    load = pickle.load(handle)
    initial_states = load['2020-03-17']

In [ ]:
##########################
## Initialize the model ##
##########################

# Initiate model with initial states, defined parameters, and proper time dependent functions
model = COVID19_SEIQRD_spatial_stratified_rescaling(initial_states, params, spatial='prov',
                        time_dependent_parameters={'Nc' : policy_function,
                                                'Nc_work' : policy_function_work,
                                                'place' : mobility_function,
                                                'f_VOC' : VOC_function,
                                                'E_susc' : E_susc_function,
                                                'E_inf' : E_inf_function,
                                                'E_hosp' : E_hosp_function,
                                                'seasonality' : seasonality_function})

# model = COVID19_SEIQRD_spatial_stratified_rescaling(initial_states, params, spatial='prov',
#                         time_dependent_parameters={'Nc' : policy_function,
#                                                 'Nc_work' : policy_function_work,
#                                                 'place' : mobility_function,
#                                                 'f_VOC' : VOC_function,
#                                                 'seasonality' : seasonality_function})

In [ ]:
###################
## Run the model ##
###################
end_visualization = '2022-06-01'
start_calibration = '2020-03-18'
warmup = 0
out = model.sim(end_visualization,start_date=start_calibration,warmup=warmup)

# Difference between JIT and simplistic (non-JIT) looping: factor 6
# Difference between rescaling TDPF and no rescaling TDPF: almost nothing

In [ ]:
out['H_in'].sum(dim='Nc').sum(dim='place').plot()

# out['H_in'].sel(time=pd.Timestamp(2022, 1, 1)).sum(dim='Nc').sum(dim='place')

In [ ]:
E_hosp_function(pd.Timestamp(2021, 1, 1), 0, 0)

In [ ]:
# df_test.groupby(['date', 'NIS', 'age', 'dose']).first().loc[:,80000, pd.Interval(45,55,closed='left'), 'weighted_sum']['E_hosp'].plot()

first_name = 'weighted_'
name = 'sum'
df_test.groupby(['date', 'NIS', 'age', 'dose']).first().loc[:,80000, pd.Interval(45,55,closed='left'), f'{first_name}{name}']

In [ ]:
# df_test2 = df_test.groupby(['date', 'NIS', 'age', 'dose']).first()
# df_test2.loc[('2020-12-28',10000,pd.Interval(0,12,closed='left'), 'weighted_sum'), 'E_susc'] = (df_test2.loc[('2020-12-28',10000,pd.Interval(0,12,closed='left')), 'E_susc'] * df_test2.loc[('2020-12-28',10000,pd.Interval(0,12,closed='left')), 'fraction']).sum()
# df_test2.groupby(['date', 'NIS', 'age', 'dose']).first()

initN, Nc_dict, params, CORE_samples_dict = model_parameters.get_COVID19_SEIQRD_parameters(spatial='prov')

In [ ]:
def waning_exp_delay(days, onset_days, E_init, E_best, E_waned):
    """
    Function that implements time-dependence of vaccine effect.

    Input
    -----
    days : float
        number of days after the novel vaccination
    onset_days : float
        number of days it takes for the vaccine to take full effect
    E_init : float
        vaccine-related rescaling value right before vaccination
    E_best : float
        rescaling value related to the best possible protection by the currently injected vaccine
    E_waned : float
        rescaling value related to the vaccine protection after a waning period.

    Output
    ------
    E_eff : float
        effective rescaling value associated with the newly administered vaccine

    """
    waning_days = 183 # hard-coded to half a year
    if days <= 0:
        return E_init
    elif days < onset_days:
        E_eff = (E_best - E_init)/onset_days*days + E_init
        return E_eff
    else:
        if E_best == E_waned:
            return E_best
        halftime_days = waning_days - onset_days
        A = 1-E_best
        beta = -np.log((1-E_waned)/A)/halftime_days
        E_eff = -A*np.exp(-beta*(days-onset_days))+1
    return E_eff

In [ ]:
def E_eff_func(df_inc, onset_days, E_init, E_best, E_waned):
    """
    Currently implemented for weekly data. Creates DataFrame with rescaling values for one particular VOC.
    
    Input
    -----
    
    t : pd.Timestamp
        Time at which we want to know the effective rescaling factor
    df_inc : pd.DataFrame
        DataFrame containing all incidence data. Output from df_inc = make_vaccination_function(vacc_data['INCIDENCE']).df
    onset_days : dict
        dict containing onset_days values for any rescaling type {'E_susc', 'E_inf', 'E_hosp'} and every vaccine stage {'first', 'full', 'booster'}
    E_init : dict
        dict containing initial rescaling values for any rescaling type {'E_susc', 'E_inf', 'E_hosp'} and every vaccine stage {'first', 'full', 'booster'}
    E_best : dict
        dict containing best rescaling values for any rescaling type {'E_susc', 'E_inf', 'E_hosp'} and every vaccine stage {'first', 'full', 'booster'}
    E_waned : dict
        dict containing waned rescaling values for any rescaling type {'E_susc', 'E_inf', 'E_hosp'} and every vaccine stage {'first', 'full', 'booster'}
    
    Output
    ------
    E_eff : np.array
        effective rescaling factor associated with a particular VOC type and a particular vaccination dose. Dimensions [G,N]
    """
    
    ### Make proper DataFrame
    # Rename column headers
    df_inc = df_inc.rename(index={'A' : 'first', 'B' : 'full', 'C' : 'janssen', 'E' : 'booster'})
    df_inc = df_inc.reset_index().set_index('date').rename(columns={0 : 'INCIDENCE'})
    
    # Add Janssen vaccins to full dose and delete Janssen vaccins
    df_inc.loc[df_inc['dose']=='full', 'INCIDENCE'] += df_inc.loc[df_inc['dose']=='janssen', 'INCIDENCE']
    df_inc = df_inc.groupby(['date','NIS','age', 'dose']).first().drop(index='janssen', level=3)
    
    # Add column with cumulative values
    df_inc['CUMULATIVE'] = df_inc.groupby(level=[1,2,3]).cumsum()
    
    ### Create column with rescaling value calculated from cum and inc values
    
    # initialise
    df_inc['E_susc'] = 0
    df_inc['E_inf'] = 0
    df_inc['E_hosp'] = 0

    # 
    df_inc = df_inc.reset_index()
    all_available_dates = df_inc.date.unique()
    df_inc = df_inc.set_index(['date', 'dose']).sort_index()
    for rescaling in ['E_susc', 'E_inf', 'E_hosp']:
        for dose in ['first', 'full', 'booster']:
            # Calculate E values for this rescaling type and dose
            onset_days_temp = onset_days[rescaling][dose]
            E_init_temp = E_init[rescaling][dose]
            E_best_temp = E_best[rescaling][dose]
            E_waned_temp = E_waned[rescaling][dose]
            for date in all_available_dates:
                # run over all dates before this date
                for d in all_available_dates[all_available_dates<=date]:
                    # Calculate how many days there are in between
                    delta_days = pd.Timedelta(date - d).days
                    # Sum over previous days with a weight depending on incidence, dose type, and waning of vaccines
                    weight = waning_exp_delay(delta_days, onset_days_temp, E_init_temp, E_best_temp, E_waned_temp)
                    df_inc.loc[(date, dose), rescaling] += df_inc.loc[(d, dose),'INCIDENCE'].to_numpy() * weight
                # normalise over total number of vaccinated subjects up to that point
                df_inc.loc[(date,dose), rescaling] /= df_inc.loc[(date,dose), 'CUMULATIVE']
        # Get rid of all division-by-zero results
        df_inc.loc[df_inc[rescaling]==np.inf, rescaling] = 1
        df_inc[rescaling].fillna(1, inplace=True)
    
    return df_inc

# test values
onset_days = dict({'E_susc' : {'first' : 14, 'full' : 14, 'booster' : 14},
               'E_inf' : {'first' : 14, 'full' : 14, 'booster' : 14},
               'E_hosp' : {'first' : 14, 'full' : 14, 'booster' : 14}})
# E_init is the value of the previous waned vaccine
E_init = dict({'E_susc' : {'first' : 1, 'full' : 0.8, 'booster' : 0.4},
               'E_inf' : {'first' : 1, 'full' : 0.5, 'booster' : 0.6},
               'E_hosp' : {'first' : 1, 'full' : 0.4, 'booster' : 0.5}})
E_best = dict({'E_susc' : {'first' : 0.4, 'full' : 0.3, 'booster' : 0.3},
               'E_inf' : {'first' : 0.5, 'full' : 0.4, 'booster' : 0.4},
               'E_hosp' : {'first' : 0.2, 'full' : 0.1, 'booster' : 0.1}})
E_waned = dict({'E_susc' : {'first' : 0.4, 'full' : 0.5, 'booster' : 0.3},
               'E_inf' : {'first' : 0.5, 'full' : 0.6, 'booster' : 0.4},
               'E_hosp' : {'first' : 0.2, 'full' : 0.2, 'booster' : 0.1}})

df_result = E_eff_func(0, df_inc, onset_days, E_init, E_best, E_waned)

In [ ]:
# df_result.loc[:,'booster',10000, pd.Interval(45,55,closed='left')]['E_hosp']
# df_result.groupby(['date', 'dose', 'NIS', 'age']).first().loc[:,'booster', 10000, pd.Interval(45,55,closed='left')]['E_hosp'].plot()

df_result.groupby(['date', 'NIS', 'age', 'dose']).first()

In [ ]:
df_temp = E_eff_func(0, df_inc, 0, 0)
df_temp = df_temp.reset_index().set_index(['date', 'dose'])
df_temp.loc[('2020-12-28', 'booster'), 'E_inf'] = 1
df_temp

In [ ]:
# case 1: cumsum = 0, so value doesn't contribute in overall sum. Return 1 (won't contribute)

# case 2: cumsum != 0, so value does contribute, weighted by how many subjects have been vaccinated yet

    
# df_cum_new = df_inc_new.groupby(level=[1,2,3]).cumsum()

# dates = df_inc_new.reset_index().date.unique()
# date = pd.Timestamp('2021, 05, 10')
# for d in dates[dates<=date]:

df_inc_temp = E_eff_func(0, df_inc, 0, 0)
df_new = df_inc_temp.reset_index()
all_available_dates = df_new.date.unique()
df_new = df_new.set_index(['date', 'dose'])
for rescaling in ['E_susc', 'E_inf', 'E_hosp']:
    for dose in ['first', 'full', 'booster']:
        for date in all_available_dates:
            # run over all dates before this date
            for d in dates[dates<=date]:
                # Calculate how many days there are in between
                delta_days = pd.Timedelta(date - d).days
                # Sum over previous days with a weight depending on incidence, dose type, and waning of vaccines
                df_new.loc[(date, dose), rescaling] += df_new.loc[(d, dose),'INCIDENCE'].to_numpy() * waning_exp_delay(delta_days, 21, 0.8, 0.2, 0.5)
            # normalise over total number of vaccinated subjects up to that point
            df_new.loc[(date,dose), rescaling] /= df_new.loc[(date,dose), 'CUMULATIVE']
    # Get rid of all division-by-zero results
    df_new.loc[df_new[rescaling]==np.inf, rescaling] = 1
    df_new[rescaling].fillna(1, inplace=True)
df_new

In [ ]:
df_temppp = df_new.drop(columns=['INCIDENCE', 'CUMULATIVE'])
df_temppp = df_temppp.reset_index().set_index(['date', 'NIS', 'age', 'dose'])
df_temppp.loc[:,10000, pd.Interval(45,55,closed='left'), 'booster']['E_susc'].plot()

In [ ]:
# df_new.loc[:, 10000, pd.Interval(25,35), 'full']

# df_new['E'].fillna(1, inplace=True)
df_new.groupby(['date', 'NIS', 'age', 'dose']).first().loc[:, 10000, pd.Interval(25,35,closed='left'), 'full']['E'].plot()

In [ ]:
df_new = df_new.groupby(['date', 'NIS', 'age', 'dose']).first()

df_new

In [ ]:

intervals = pd.IntervalIndex.from_tuples([(0,12),(12,18),(18,25),(25,35),(35,45),(45,55),(55,65),(65,75),(75,85),(85,120)], closed='left')
intervals_str = np.array(['[0, 12)', '[12, 18)', '[18, 25)', '[25, 35)', '[35, 45)', '[45, 55)', '[55, 65)', '[65, 75)', '[75, 85)', '[85, 120)'])
intervals_dict = dict({intervals_str[i] : intervals[i] for i in range(len(intervals))})
initN = initN.rename(columns=intervals_dict)

# initN_fulltable = initN.unstack().rename(column=dict({'age_class' : 'age'}))

df_new = pd.DataFrame(vacc_function.df).reset_index()
df_new = df_new.merge(initN.unstack().reset_index(), left_on=['NIS', 'age'], right_on=['NIS', 'age_class'])
df_new['fraction'] = df_new['0_x'] / df_new['0_y']
# df_new = df_new.drop(columns=['age_class', '0_x', '0_y'])
# df_new = df_new.set_index(['date', 'NIS', 'age', 'dose'])

# df_new.loc[:,:,:,'B']['fraction'] = df_new.loc[:,:,:,'A']['fraction'] - df_new.loc[:,:,:,'B']['fraction']
# df_new.loc[:,:,:,'A']['fraction']

df_new = df_new.set_index('date') # make sure we don't get NaN values because of mismatching indices
df_new_copy = df_new.copy()

# first-only: dose A (first) - dose B (second)
df_new.loc[df_new['dose']=='B','fraction'] = (df_new_copy.loc[df_new_copy['dose']=='A','fraction'] \
    - df_new_copy.loc[df_new_copy['dose']=='B','fraction']).clip(lower=0, upper=1)

# full: dose B (second) + dose C (Jansen) - dose E (booster)
df_new.loc[df_new['dose']=='C','fraction'] = (df_new_copy.loc[df_new_copy['dose']=='B','fraction'] \
    + df_new_copy.loc[df_new_copy['dose']=='C','fraction'] - df_new_copy.loc[df_new_copy['dose']=='E','fraction']).clip(lower=0, upper=1)

# booster: clip between 0 and 1. This is currently the latest stage
df_new.loc[df_new['dose']=='E','fraction'] = df_new_copy.loc[df_new_copy['dose']=='E', 'fraction'].clip(lower=0, upper=1)

# none. Rest category. Make sure all adds up to 1.
df_new.loc[df_new['dose']=='A','fraction'] = 1 - (df_new_copy.loc[df_new_copy['dose']=='B','fraction'] \
    - df_new_copy.loc[df_new_copy['dose']=='C','fraction'] - df_new_copy.loc[df_new_copy['dose']=='E','fraction']).clip(lower=0, upper=1)

# Return to multiindex
df_new = df_new.reset_index()
df_new = df_new.drop(columns=['age_class', '0_x', '0_y'])
df_new = df_new.set_index(['date','NIS', 'age', 'dose'])

# rename indices to clearly understandable categories
rename_indices = dict({'A' : 'none', 'B' : 'first', 'C' : 'full', 'E' : 'booster'})
df_new = df_new.rename(index=rename_indices)

# df_new_copy = df_new.copy()


In [ ]:
make_vaccination_function(vacc_data['CUMULATIVE']).df

In [ ]:
vacc_data.rename(index=dict({10000 : 'test'}))

                                                                                                                     
# initN

# initN.rename(columns=intervals_dict)

initN

In [ ]:
import datetime

df = pd.read_csv('../../data/raw/sciensano/COVID19BE_VACC_MUNI_raw.csv')

start=[]
end=[]
for year_week in df["YEAR_WEEK"].values:
    year = '20'+year_week[0:2]
    week = year_week[3:]
    if week == '53':
        week = str(int(year_week[3:])-1)

    startdate = "{}-{}-1".format(year, week)
    enddate = "{}-{}-6".format(year, week)

    dt = datetime.datetime.strptime(startdate, "%Y-%W-%w")
    start.append(dt.strftime("%Y-%m-%d"))

    dt = datetime.datetime.strptime(enddate, "%Y-%W-%w")
    end.append(dt.strftime("%Y-%m-%d"))
df['start_week'] = start
df['end_week'] = end

df

In [ ]:
######################
## Format dataframe ##
######################

df = df.drop(df[df.NIS5 == ''].index)
df = df.dropna()
df['CUMUL'][df['CUMUL'] == '<10'] = '0'
df['CUMUL'] = df['CUMUL'].astype(int)
df['NIS5'] = ((df['NIS5'].astype(float)).astype(int)).astype(str)
df = df.rename(columns={'NIS5':'NUTS5', 'AGEGROUP':'age'})
df.set_index('start_week')
df.pop('YEAR_WEEK')
df.pop('end_week')
df.set_index(['start_week','NUTS5','age','DOSE'], inplace=True)

In [ ]:
pd.concat?

In [ ]:
vacc_data